import packages & selct dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import os
import h5py
import importlib
import glob

import mosaic_topog.utilities as util
import mosaic_topog.flsyst as flsyst
import mosaic_topog.show as show
import mosaic_topog.calc as calc

data_path = 'C:\\Users\\schle\Documents\\GitHub\\Incubator-2022-Geometry-of-Color\\CSV data all OCT locs Jan 2022\\'
save_path = 'C:\\Users\\schle\Documents\\Incubator Data\\'

subject = np.asarray(['AO001R', 'AO008R'])
angle = np.asarray(['temporal', 'nasal', 'superior', 'inferior'])
eccentricity = np.asarray(['1.5', '4', '10'])

coord_fltype = '.csv'
img_fltype = '.png'

dist_unit = 'pixels'

filetypes = [coord_fltype, img_fltype]

conetype_ids = ['all','L','M','S','unclassed']

conetype_colors = ['y','r','g','b','k']
conetype_sizes = [10, 10, 10, 30]

bin_width = 5

num_mc = 100 # number of monte carlo mosaics to generate

categories = [subject, angle, eccentricity]

load in coordinate data and images

In [ ]:
[coord_data, coord_mosaics, coord_index] = flsyst.getConeData(data_path,subject,angle,eccentricity,conetype_ids,filetypes[0])
[img_data, img_mosaics, img_index] = flsyst.getConeData(data_path,subject,angle,eccentricity,conetype_ids,filetypes[1])


plot cones on the image 

In [ ]:
# for each mosaic we have an image of...
axes = []
for ind,mos in enumerate(img_mosaics):
    # get the image
    img = img_data[ind]
    
    # find the same mosaic in our coordinate data 
    # (may or may not be represented by the same number) 
    coord_mos_ind = coord_mosaics.index(mos)
    
    # pull all coone coordinate filse associated with this mosaic
    coord_mos = flsyst.getDataByCatVal(coord_data, coord_index['mosaics'], coord_mos_ind)
    
    # make a dictionary of all coordinate files and their associated conetype_ids
    coords = {}
    for ind,id in enumerate(conetype_ids):
        coords[id] = coord_mos[ind]
    
    axes.append(show.plotOnROI(img, coords, conetype_ids, mos, conetype_colors, title=mos, figsize=10))
    

reimport packages in dev, reload in the mosaic data

In [16]:
importlib.reload(flsyst)
importlib.reload(calc)

[data, mosaics, index, flnames] = flsyst.getConeData(data_path,subject,angle,eccentricity,conetype_ids,filetypes[0])

found 110 files



run checks for files that need to have calculations run on them

In [17]:
dist_group = 'intracone_dists'
dist_keys = ['cone_dists_square','cone_dists_vect','avg_nearest_cone','std_nearest_cone','dist_unit']

collect_all = [] # indices of data files that need all operations run on them
collect_dists = [] # indices of data files that need intercone distance calculations run 

# Make list of expected save-file names for this data
savenames = []
for ind,fl in enumerate(data):
    savenames.append(save_path + flnames[ind] + '.hdf5')
    
# look for files associated with each data file in the save folder
# If no data file is found, add it to the list to collect all data for
folder_hdf5s = glob.glob(save_path + '*.hdf5') # get all .hdf5 files in the save path
collect_all = util.indsNotInList(savenames,folder_hdf5s)

# If data file is found, look for the associated keys. If they are not found, add to the list
savenames = np.array(savenames)
for ind,fl in enumerate(data):
    if ind not in collect_all:
         temp = h5py.File(savenames[ind], 'r')
         #check intercone distance related keys
         for k in dist_keys:
            if k not in temp.keys():
                collect_dists.append(ind)
collect_dists = np.array(collect_dists)

calculate the distances between all cones in a mosaic and save them

In [19]:
to_calc = np.union1d(collect_all, collect_dists).astype(int)

for ind in to_calc:
    dists = calc.dist_matrices(data[ind])
    
    # get avg and std of nearest cone distance in the mosaic
    nearest_dist = []
    
    for cone in np.arange(0,dists.shape[0]):
        # get row for this cone's distance to every other cone
        row = dists[cone,:]
        # find the index where the distance = -1 if it exists - this is self and shouldn't be included
        row = np.delete(row,np.nonzero(row==-1))
        # get the minimum value in the row
        nearest_dist.append(row.min())
    
    mean_nearest = np.mean(np.array(nearest_dist))
    std_dist = np.std(np.array(nearest_dist))
    
    # start a save file
    savename = savenames[ind]
    sav = h5py.File(savename, 'w')
    group = sav.create_group('intracone_dists')
    group['cone_dists_square'] = square.astype(np.float64)
    group['avg_nearest_cone'] = mean_nearest.astype(np.float64)
    group['std_nearest_cone'] = std_nearest.astype(np.float64)
    group['dist_unit'] = dist_unit


FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = 'C:\Users\schle\Documents\Incubator Data\AO001R_temporal_1.5_all.csv.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [4]:
collect_dists
np.union1d(collect_all,collect_dists)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.])